<h1> GENRE: POP </h1>

<h3>(1) Access the data from the dataset </h3>


In [1]:
%%capture
%pip install datasets

from datasets import load_dataset

ds = load_dataset("sebastiandizon/genius-song-lyrics")

In [ ]:
import heapq
from datasets import Dataset
type_of_music= 'rock'
TOP_K = 1000
decade_heaps = {70: [], 80: [], 90: [], 00: [], 10:[], 20:[]}  # z.B. 70=70er, 0=00er etc.

def get_decade(year):
    if 1970 <= year <= 1979:
        return 70
    elif 1980 <= year <= 1989:
        return 80
    elif 1990 <= year <= 1999:
        return 90
    elif 2000 <= year <= 2009:
        return 00
    elif 2010 <= year <= 2019:
        return 10
    elif 2020 <= year <= 2029:
        return 20
    return None

i = 0  # Zähler für tie-break
for song in ds["train"]:
    if song["tag"] != type_of_music:
        continue
    elif song["language"] != "en":
        continue
    dec = get_decade(song["year"])
    if dec is None:
        continue

    views = song["views"]
    heap = decade_heaps[dec]

    # Das Tupel enthält jetzt (views, i, song).
    # Der zweite Wert i ist rein für Tie-Break (und garantiert sortierbar).
    if len(heap) < TOP_K:
        heapq.heappush(heap, (views, i, song))
    else:
        # Vergleiche nur, ob aktuelle views (oder Tie-Break i) größer sind 
        # als das Minimum im Heap (heap[0]).
        if (views, i) > (heap[0][0], heap[0][1]):
            heapq.heapreplace(heap, (views, i, song))
    i += 1

# Am Ende hat jede Dekade max. 1000 Elemente.
decade_toplists = {}
for dec, heap in decade_heaps.items():
    # Sortieren nach (views, i) absteigend.
    # Falls dir der Index egal ist, kannst du einfach nur nach views sortieren.
    sorted_heap = sorted(heap, key=lambda x: (x[0], x[1]), reverse=True)
    # x[0] = views, x[1] = tie-break index, x[2] = song
    songs_only = [item[2] for item in sorted_heap]
    decade_toplists[dec] = songs_only

# Optional: zu HF Dataset konvertieren
subdataset_70 = Dataset.from_list(decade_toplists[70])
subdataset_80 = Dataset.from_list(decade_toplists[80])
subdataset_90 = Dataset.from_list(decade_toplists[90])
subdataset_00 = Dataset.from_list(decade_toplists[00])
subdataset_10 = Dataset.from_list(decade_toplists[10])
subdataset_20 = Dataset.from_list(decade_toplists[20])

path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/"
# Save to hard drive
subdataset_70.save_to_disk(path + "/Subdatasets/subdataset_70_rock")
subdataset_80.save_to_disk(path + "/Subdatasets/subdataset_80_rock")
subdataset_90.save_to_disk(path + "/Subdatasets/subdataset_90_rock")
subdataset_00.save_to_disk(path + "/Subdatasets/subdataset_00_rock")
subdataset_10.save_to_disk(path + "/Subdatasets/subdataset_10_rock")
subdataset_20.save_to_disk(path + "/Subdatasets/subdataset_20_rock")

PermissionError: [Errno 13] Permission denied: '/Subdatasets'

In [4]:
path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/"
# Save to hard drive
subdataset_70.save_to_disk(path + "/Subdatasets/rock/subdataset_70_rock")
subdataset_80.save_to_disk(path + "/Subdatasets/rock/subdataset_80_rock")
subdataset_90.save_to_disk(path + "/Subdatasets/rock/subdataset_90_rock")
subdataset_00.save_to_disk(path + "/Subdatasets/rock/subdataset_00_rock")
subdataset_10.save_to_disk(path + "/Subdatasets/rock/subdataset_10_rock")
subdataset_20.save_to_disk(path + "/Subdatasets/rock/subdataset_20_rock")

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

<h2> LDA </h2>

<h2> Remove the bad words</h2>

In [1]:
subdataset_00[1]

NameError: name 'subdataset_00' is not defined

In [2]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("tushifire/ldnoobw")
path = kagglehub.dataset_download("sahib12/badwords")

#file = kagglehub.load_dataset("tushifire/ldnoobw",path)

print("Path to dataset files:", path)

swear_words_file = os.path.join(path, 'Terms-to-Block.csv')  # Replace 'en.txt' with the correct filename

with open(swear_words_file, 'r') as f:
    swear_words = set(f.read().splitlines())

print(f"!!!!Loaded {len(swear_words)} swear words.")

#print(swear_words)
print("Done")


Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
Done


In [6]:
# import gensim
# import re
# from gensim.corpora.dictionary import Dictionary
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# import nltk
# from gensim.parsing.preprocessing import remove_stopwords

# # Sicherstellen, dass NLTK-Daten heruntergeladen sind
# nltk.download('stopwords')
# nltk.download('punkt')

# # Beispiel-Liste von Strings


# # Stopwords definieren
# stop_words = set(stopwords.words('english')+["like", "oh", "na", "la", "yo", "get", "yeah", "got","verse", "chorus", "hook", "bridge", "outro", "intro"])
# print("Stopwords:", stop_words)
# # Preprocessing: Tokenisieren, Stopwords entfernen, Kleinbuchstaben
# def preprocess(doc):
#     doc = re.sub(r"[^a-zA-Z0-9\s]", " ", doc) # delete unncessary things
#     #doc = remove_stopwords(doc.lower())
#     doc = remove_repeated_lines(doc)
    
#     tokens = word_tokenize(doc.lower())  # Kleinbuchstaben und Tokenisierung
    
#     tokens = [word for word in tokens if word not in stop_words]  # Stopwords entfernen
    
#     return tokens

# def remove_repeated_lines(doc):
#     lines = doc.split("\n")
#     return "\n".join(sorted(set(lines), key=lines.index))

# def sort_out_bad_words(doc,word_list, replacement=" "):
#     """
#     Ersetzt Wörter in einem Text, wenn sie in einer Liste vorkommen.
    
#     :param text: Der Eingabetext (String).
#     :param word_list: Liste der Wörter, die ersetzt werden sollen.
#     :param replacement: Der Text, durch den die Wörter ersetzt werden sollen.
#     :return: Der bearbeitete Text.
#     """
#     # Erstelle ein Set für einen schnelleren Lookup
#     word_set = set(word_list)
    
#     # Splitte den Text in Wörter
#     words = doc.split()
    
#     # Ersetze jedes Wort, falls es in der Liste vorkommt
#     replaced_words = [replacement if word.lower() in word_set else word for word in words]
    
#     # Füge den Text wieder zusammen
#     return " ".join(replaced_words)
 

# def process_subdatasets(subdataset):
#     # Get rid of the bad words and replace it with "swear word"
#     without_swear = [sort_out_bad_words(doc, swear_words ) for doc in subdataset]

#     # Preprocessing für alle Dokumente anwenden
#     processed_docs = [preprocess(doc) for doc in without_swear]

#     if stop_words in processed_docs:
#         print("FEHLER: Stopwords nicht entfernt!")
        
#     # Erstellen eines Wörterbuchs
#     dictionary = Dictionary(processed_docs)

#     # Erstellen eines Bag-of-Words-Korpus
#     corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

#     # LDA-Modell trainieren
#     lda_model = gensim.models.LdaModel(
#         corpus=corpus,
#         id2word=dictionary,
#         num_topics=5,  # Anzahl der Themen
#         random_state=42,
#         passes=40,  # Anzahl der Durchgänge
#         alpha='symmetric',
#         eta='auto'
#         #alpha='auto'  # Dirichlet-Hyperparameter Entscheidung zwischen Auto und symetric
#     )
    
#     id2word = dictionary
#     dict_topic = {}
#     # Themen anzeigen
#     #print("Themen und ihre Schlüsselwörter:")
#     for idx, topic in lda_model.print_topics(num_words=5):
#         #print(f"Topic {idx}: {topic}")
#         dict_topic[idx] = topic 

#     return dict_topic


Stopwords: {"weren't", 'both', 'bridge', 'don', 'won', 've', "you'd", 'he', 'him', 'been', "hadn't", 'weren', 'once', 'doesn', 'other', "you'll", 'wasn', 't', 'have', 'until', 'when', 'will', 'through', "don't", 'o', 'didn', 'wouldn', 'chorus', 'aren', 'below', 'nor', 'about', 'needn', 'not', "isn't", "mightn't", 'an', 'get', 'because', 'shouldn', 'being', 'like', 'down', 'they', 'than', 'myself', 'own', 'yeah', 'haven', 'ours', 'shan', 'under', 'oh', 'all', 'can', 'y', "that'll", "aren't", 'na', 'verse', 'its', 'where', 'at', 'my', 'more', 'same', 'very', "needn't", 'are', 'for', 'intro', 'between', 'most', 'm', "shouldn't", 'couldn', "couldn't", 'why', 'themselves', 'from', "shan't", "wouldn't", 'but', 'yourselves', 'before', 'did', 'a', 'his', "hasn't", 'got', "you've", 'how', 'some', 'yo', 'la', 'outro', 'who', 'such', 'to', 'those', 'does', "you're", 'during', 'ain', 'you', "mustn't", "haven't", 'whom', 'having', "wasn't", 'were', 'only', 'be', 'then', "didn't", 'am', 'after', 'd'

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maurice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/maurice/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import gensim
import re
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from gensim.parsing.preprocessing import remove_stopwords

# Sicherstellen, dass NLTK-Daten heruntergeladen sind
nltk.download('stopwords')
nltk.download('punkt')

used_swear_words = []
# clean the text
import re


# Stopwords definieren
stop_words = set(stopwords.words('english')+["like", "oh", "na", "la", "yo", "get", "yeah", "got","verse", "chorus", "hook", "bridge", "outro", "intro"])
print("Stopwords:", stop_words)

def clean_text(document_list):
    
    cleaned_document_list = []
    
    for single_doc in document_list:
        # Delete HTML-Tags
        text = re.sub(r"<.*?>", "", single_doc)
        # Delete Sonderzeichen und Zahlen
        text = re.sub(r"[^a-zA-Z\s]", "", text)
        # Delete uncessary whitespaces
        text = re.sub(r"\s+", " ", text).strip()
        
        # lower case the text
        text = text.lower()
        
        cleaned_document_list.append(text)
    
    return cleaned_document_list


def lemmatize_docs(subdataset):
    import spacy

    nlp = spacy.load("en_core_web_md")
    
    lemmatized_document_list = []
    lemmatized = []
    
    for single_doc in subdataset:
        nlp_doc = nlp(single_doc)

        for token in nlp_doc:
        # Nur Tokens behalten, die:
        # - kein Stopwort sind
        # - kein Satzzeichen sind
        # - eine gewisse Länge haben (z. B. > 1 Zeichen)
            if not token.is_stop and not token.is_punct and len(token.text.strip()) > 1:
                lemmatized.append(token.lemma_.lower())
        
        
        lemmatized_document_list.append(lemmatized)
    
    return lemmatized_document_list
    

def get_swear_words():

    path = kagglehub.dataset_download("tushifire/ldnoobw")
    path = kagglehub.dataset_download("sahib12/badwords")

    #file = kagglehub.load_dataset("tushifire/ldnoobw",path)

    print("Path to dataset files:", path)

    swear_words_file = os.path.join(path, 'Terms-to-Block.csv')  # Replace 'en.txt' with the correct filename

    with open(swear_words_file, 'r') as f:
        swear_words = set(f.read().splitlines())

    print(f"!!!!Loaded {len(swear_words)} swear words.")

    return swear_words

def get_manual_stop_words():
    
    path = '/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Manually_Banned_Words.txt'
    path_manually_stop_words = '/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/stopwords_en.txt'

    with open(path, 'r', encoding='utf-8') as f:
        text_banned = f.read()
    
    with open(path_manually_stop_words, 'r', encoding='utf-8') as f:
        text_stopwords = f.read()
        
    banned_words = set(text_banned.split())
    stop_words = set(text_stopwords.split())
    
    return banned_words.union(stop_words)

# Preprocessing: Tokenisieren, Stopwords entfernen, Kleinbuchstaben
def preprocess_stop_words(document_list):
    # Stopwords definieren
    
    stop_words = set(stopwords.words('english'))  # Standard-Stopwords
    #additional_words = {"like", "oh", "na", "la", "yo", "you", "get", "yeah" 
    #                       "the", "and", "of", "on", "[Intro]", "[Storyteller]"}
    
    # import the manual files of stop words
    additional_words = get_manual_stop_words()
    swear_words = get_swear_words()
      
    word_set = stop_words.union(additional_words)
    word_set = word_set.union(swear_words)
        
    replacement = ""
    list_of_tokens = []
    for single_doc in document_list:

        words = single_doc.split()
        
        # replace word if not in the list
        replaced_words = []
        for word in words:
            if word.lower() in word_set:
                replaced_words.append(replacement)  
                
                if word.lower() in swear_words:
                    used_swear_words.append(word) # save word to the used swear words list
            else:
                replaced_words.append(word)  
                
        list_of_tokens.append(str(" ".join(replaced_words)))
              
    
    return list_of_tokens

def remove_repeated_lines(doc):

    unique_lines = []
    seen = set()
    for line in doc:
        if line not in seen:
            unique_lines.append(line)
            seen.add(line)
    # Am Ende kannst du wieder eine Liste zurückgeben:
    return unique_lines

def tokenize_all_docs(document_list):
    list_of_tokens = []
    
    for single_doc in document_list:
        
        if type(single_doc) == str:
            tokens = word_tokenize(single_doc)
            list_of_tokens.append(tokens)
        else:
            continue
    
    return list_of_tokens

def get_used_swear_words():
    return used_swear_words




def preprocess(doc):
    doc = re.sub(r"[^a-zA-Z0-9\s]", " ", doc) # delete unncessary things
    #doc = remove_stopwords(doc.lower())
    doc = remove_repeated_lines(doc)
    
    tokens = word_tokenize(doc.lower())  # Kleinbuchstaben und Tokenisierung
    
    tokens = [word for word in tokens if word not in stop_words]  # Stopwords entfernen
    
    return tokens

def remove_repeated_lines(doc):
    lines = doc.split("\n")
    return "\n".join(sorted(set(lines), key=lines.index))

def sort_out_bad_words(doc,word_list, replacement=" "):

    # Erstelle ein Set für einen schnelleren Lookup
    word_set = set(word_list)
    
    # Splitte den Text in Wörter
    words = doc.split()
    
    # Ersetze jedes Wort, falls es in der Liste vorkommt
    replaced_words = [replacement if word.lower() in word_set else word for word in words]
    
    # Füge den Text wieder zusammen
    return " ".join(replaced_words)
 

def process_subdatasets(subdataset):
    # Get rid of the bad words and replace it with "swear word"
    #without_swear = [sort_out_bad_words(doc, swear_words ) for doc in subdataset]

    # Preprocessing für alle Dokumente anwenden
    #processed_docs = [preprocess(doc) for doc in without_swear]

    cleaned_dataset_no_strange_characters = clean_text(subdataset)

    cleaned_dataset_no_swears = preprocess_stop_words(cleaned_dataset_no_strange_characters)

    lemmatized_dataset = lemmatize_docs(cleaned_dataset_no_swears)
    
    #tokenized_dataset = tokenize_all_docs(lemmatized_dataset)


    dictionary = Dictionary(lemmatized_dataset)
    
    # Erstellen eines Bag-of-Words-Korpus
    corpus = [dictionary.doc2bow(doc) for doc in lemmatized_dataset]

    # LDA-Modell trainieren
    lda_model = gensim.models.LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=5,  # Anzahl der Themen
        random_state=42,
        passes=40,  # Anzahl der Durchgänge
        alpha='symmetric',
        eta='auto'
        #alpha='auto'  # Dirichlet-Hyperparameter Entscheidung zwischen Auto und symetric
    )
    
    id2word = dictionary
    dict_topic = {}
    # Themen anzeigen
    #print("Themen und ihre Schlüsselwörter:")
    for idx, topic in lda_model.print_topics(num_words=5):

        dict_topic[idx] = topic 

    return dict_topic


Stopwords: {'bridge', 'so', "needn't", 'but', 'his', 'few', 'get', 'won', "hasn't", "it's", 'against', 'now', "you'd", 'yo', 'couldn', 'didn', 'they', 'mightn', 'the', 'or', 'aren', 'because', "won't", 've', 'mustn', 'under', 'in', 'having', 'why', 'were', 'not', 'and', 're', 'themselves', 'there', 'when', 't', 'which', 'its', 'before', 'are', 'here', 'just', 'ourselves', 'of', "didn't", 'we', 'once', 'did', 'being', 'she', 'up', 'is', 'very', 'wouldn', 'too', 'hook', 'again', 'during', 'her', "shan't", 'don', "wouldn't", 'your', 'any', 'hadn', 'be', 'intro', 's', 'isn', 'can', 'other', 'all', "hadn't", 'like', 'nor', "you've", 'above', 'yours', 'd', 'off', "isn't", 'with', "haven't", 'than', 'hasn', 'from', 'chorus', 'each', 'on', 'wasn', 'na', 'those', 'needn', 'to', 'am', "weren't", 'down', 'how', 'himself', 'll', 'haven', "wasn't", 'yourself', 'herself', 'own', "mightn't", 'ours', 'myself', 'at', 'most', 'o', 'outro', 'our', 'him', 'no', 'hers', 'over', 'ain', 'shan', 'has', 'furth

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maurice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/maurice/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
amount = 1000
from datasets import load_from_disk

path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Subdatasets/rock/"


rock_70_lyrics = process_subdatasets(load_from_disk(path + 'subdataset_70_rock')['lyrics'][:amount])
rock_80_lyrics = process_subdatasets(load_from_disk(path + "subdataset_80_rock")['lyrics'][:amount])
rock_90_lyrics = process_subdatasets(load_from_disk(path + "subdataset_90_rock")['lyrics'][:amount])
rock_00_lyrics = process_subdatasets(load_from_disk(path + "subdataset_00_rock")['lyrics'][:amount])
rock_10_lyrics = process_subdatasets(load_from_disk(path + "subdataset_10_rock")['lyrics'][:amount])
rock_20_lyrics = process_subdatasets(load_from_disk(path + "subdataset_20_rock")['lyrics'][:amount])

list_of_lyrics = {
    "70s": rock_70_lyrics,
    "80s": rock_80_lyrics,
    "90s": rock_90_lyrics,
    "00s": rock_00_lyrics,
    "10s": rock_10_lyrics,
    "20s": rock_20_lyrics
}

import csv

data_for_csv = []

for decade_name, decade in list_of_lyrics.items():
    i = 0
    for topics in decade:
        i = i + 1
        data_for_csv.append({
            "Method": "LDA",
            "Genre": "Rock",
            "Decade": decade_name,
            "Topic": str(decade[topics])
        })

path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/"
# Datei schreiben
output_file = path + "Analysis_Folder/LDA/topics_rock_LDA.csv"
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Method", "Genre", "Decade", "Topic"])
    writer.writeheader()  
    writer.writerows(data_for_csv) 

print(f"CSV-Datei '{output_file}' wurde erfolgreich erstellt!")


FileNotFoundError: [Errno 2] No such file or directory: '/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Subdatasets/rock/Analysis_Folder/LDA/topics_rock_LDA.csv'

In [9]:
path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/"
output_file = path + "Analysis_Folder/topics_rock_LDA.csv"
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Method", "Genre", "Decade", "Topic"])
    writer.writeheader()  
    writer.writerows(data_for_csv) 

<h3>BiGrams </h3>

In [5]:
# Input Token

from gensim.models.phrases import Phrases, Phraser
from gensim.corpora import Dictionary
from gensim.models import LdaModel


def calculate_with_bigrams(subdataset):
    # Phrases braucht eine Liste von tokenisierten Sätzen
    # mindestens in 2 Dokuemnten min_count=2
    # threshold= strikt, wie oft ein bigram vorkommen muss (höher)
    
    cleaned_dataset_no_strange_characters = clean_text(subdataset)

    cleaned_dataset_no_swears = preprocess_stop_words(cleaned_dataset_no_strange_characters)

    lemmatized_dataset = lemmatize_docs(cleaned_dataset_no_swears)
    
    #tokenized_dataset = tokenize_all_docs(lemmatized_dataset)
    tokenized_dataset=lemmatized_dataset
    
    bigram = Phrases(tokenized_dataset, min_count=1, threshold=1)


    bigram_phraser = Phraser(bigram)

    tokenized_corpus_bigrams = [bigram_phraser[sent] for sent in tokenized_dataset]
    #for sent in tokenized_corpus_bigrams[:5]:
    #    print("Nach Bigram-Phraser:", sent)
    #    print("-----")


    # 1) Dictionary
    dictionary = Dictionary(tokenized_corpus_bigrams)

    # 2) Bag-of-Words
    corpus_bow = [dictionary.doc2bow(doc) for doc in tokenized_corpus_bigrams]

    # 3) LDA trainieren
    lda_model = LdaModel(corpus=corpus_bow,
                        id2word=dictionary,
                        num_topics=2,  # Beispiel, wähle was passt
                        passes=10,     # Mehrfache Durchläufe
                        random_state=42)

    # Ergebnis inspizieren
    #for idx, topic in lda_model.show_topics(formatted=False, num_words=5):
    #    print(f"Topic {idx}: {[word for word, _ in topic]}")
        
    dict_topic = {}
    for idx, topic in lda_model.print_topics(num_words=10):

        dict_topic[idx] = topic 

    return dict_topic
    

In [7]:
amount = 1000
from datasets import load_from_disk

path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Subdatasets/rock/"


rap_70_lyrics = calculate_with_bigrams(load_from_disk(path + 'subdataset_70_rock')['lyrics'][:amount])
print("Done")
rap_80_lyrics = calculate_with_bigrams(load_from_disk(path + "subdataset_80_rock")['lyrics'][:amount])
print("Done")
rap_90_lyrics = calculate_with_bigrams(load_from_disk(path + "subdataset_90_rock")['lyrics'][:amount])
print("Done")
rap_00_lyrics = calculate_with_bigrams(load_from_disk(path + "subdataset_00_rock")['lyrics'][:amount])
print("Done")
rap_10_lyrics = calculate_with_bigrams(load_from_disk(path + "subdataset_10_rock")['lyrics'][:amount])
print("Done")
rap_20_lyrics = calculate_with_bigrams(load_from_disk(path + "subdataset_20_rock")['lyrics'][:amount])
print("Completly Done")

list_of_lyrics = {
    "70s": rap_70_lyrics,
    "80s": rap_80_lyrics,
    "90s": rap_90_lyrics,
    "00s": rap_00_lyrics,
    "10s": rap_10_lyrics,
    "20s": rap_20_lyrics
}

import csv

data_for_csv = []

for decade_name, decade in list_of_lyrics.items():
    i = 0
    for topics in decade:
        i = i + 1
        data_for_csv.append({
            "Method": "BiGrams",
            "Genre": "Rock",
            "Decade": decade_name,
            "Topic": str(decade[topics])
        })
        


# Datei schreiben
path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Analysis_Folder/LDA_BiGrams/"

output_file = path + "topics_rock_LDA_1000_11_1.csv"
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Method", "Genre", "Decade", "Topic"])
    writer.writeheader()  
    writer.writerows(data_for_csv) 

print(f"CSV-Datei '{output_file}' wurde erfolgreich erstellt!")


Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
Done
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
Done
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
Done
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
Done
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
Done
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
Completly Done
CSV-Datei '/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Analysis_Folder/LDA_BiGrams/topics_rock_LDA_1000_11_1.csv' wurde erfolgreich erstellt!


# tipps von ChaGPT
https://chatgpt.com/c/676f1ffc-b134-8012-9f03-6bd466bda148

- Herausfinden, wieviele Topics am geeingesten sind
- warum sind weiterhin so viele Stop words enthalten
- wieviele Lieder nehm ich überhaupt

In [ ]:
print(len(subdataset_80))

In [ ]:
print(len(subdataset_90))

<h1> BERT TOPIC MODELLING </h1>


In [8]:
import gensim
import re
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from gensim.parsing.preprocessing import remove_stopwords
import kagglehub
import os


# Sicherstellen, dass NLTK-Daten heruntergeladen sind
nltk.download('stopwords')
nltk.download('punkt')

# clean the text
import re

def clean_text(document_list):
    
    cleaned_document_list = []
    
    for single_doc in document_list:
        # Delete HTML-Tags
        text = re.sub(r"<.*?>", "", single_doc)
        # Delete Sonderzeichen und Zahlen
        text = re.sub(r"[^a-zA-Z\s]", "", text)
        # Delete uncessary whitespaces
        text = re.sub(r"\s+", " ", text).strip()
        
        # lower case the text
        text = text.lower()
        
        cleaned_document_list.append(text)
    
    return cleaned_document_list

def get_swear_words():

    path = kagglehub.dataset_download("tushifire/ldnoobw")
    path = kagglehub.dataset_download("sahib12/badwords")

    #file = kagglehub.load_dataset("tushifire/ldnoobw",path)

    print("Path to dataset files:", path)

    swear_words_file = os.path.join(path, 'Terms-to-Block.csv')  # Replace 'en.txt' with the correct filename

    with open(swear_words_file, 'r') as f:
        swear_words = set(f.read().splitlines())

    print(f"!!!!Loaded {len(swear_words)} swear words.")

    return swear_words

# Preprocessing: Tokenisieren, Stopwords entfernen, Kleinbuchstaben
def preprocess_stop_words(document_list):
    # Stopwords definieren
    
    stop_words = set(stopwords.words('english'))  # Standard-Stopwords
    additional_words = {"like", "oh", "na", "la", "yo", "you", "get", 
                            "the", "and", "of", "on", "[Intro]", "[Storyteller]"}
    
    swear_words = get_swear_words()
      
    word_set = stop_words.union(additional_words)
    word_set = word_set.union(swear_words)
        
    replacement = ""
    list_of_tokens = []
    for single_doc in document_list:
        # Kombiniere Standard-Stopwords mit zusätzlichen Wörtern

        # Splitte den Text in Wörter
        words = single_doc.split()
        
        # Ersetze jedes Wort, falls es in der Liste vorkommt
        replaced_words = []
        for word in words:
            if word.lower() in word_set:
                replaced_words.append(replacement)  # Ersetze das Wort
            else:
                replaced_words.append(word)  # Behalte das Wort
        
        # Füge den Text wieder zusammen
        list_of_tokens.append(str(" ".join(replaced_words)))
              
    
    return list_of_tokens

def remove_repeated_lines(doc):
    # doc ist eine Liste von Strings, z.B. ["Zeile1", "Zeile2", ...]
    # Wiederholte Zeilen entfernen und Reihenfolge beibehalten
    unique_lines = []
    seen = set()
    for line in doc:
        if line not in seen:
            unique_lines.append(line)
            seen.add(line)
    # Am Ende kannst du wieder eine Liste zurückgeben:
    return unique_lines

def tokenize_all_docs(document_list):
    list_of_tokens = []
    
    for single_doc in document_list:
        
        if type(single_doc) == str:
            tokens = word_tokenize(single_doc)
            list_of_tokens.append(tokens)
        else:
            print("Type von Document", type(single_doc))
            continue
    
    return list_of_tokens

print("Done")


Done


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maurice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/maurice/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import csv


def processBertTopics(dataset,sentenceTransformer):
    i = 0
    # Initialize your topic model
    #embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
    
    embedding_model = SentenceTransformer(sentenceTransformer)
    topic_model = BERTopic(embedding_model=embedding_model)
    # Fit and transform the text data
    #without_repatition = remove_repeated_lines(dataset)
    
    # clean dataset from HTML-Tags, special characters and numbers
    cleaned_song_lyrics = clean_text(dataset)
    
    # Get rid of the bad words and replace it with NOTHING
    swear_words = set(stopwords.words('english'))
    #without_swear = [sort_out_bad_words(doc) for doc in cleaned_song_lyrics]
    
    #Deleting the stopwords
    without_stop_words = preprocess_stop_words(cleaned_song_lyrics)
    
    tokenized_docs = tokenize_all_docs(without_stop_words)
    
    # Convert 2 strings
    tokenized_docs = [" ".join(tokens) for tokens in tokenized_docs]

    # Remove empty strings
    tokenized_docs = [doc for doc in tokenized_docs if doc]
    
    # Remove documents with less than 3 words    
    #tokenized_docs = [doc for doc in tokenized_docs if len(doc.split()) > 3]
    #print("tokenized_docs", tokenized_docs)

    if tokenized_docs == [] or len(tokenized_docs) == 0:
        i = i+1
    else: 
        topics, probs = topic_model.fit_transform(tokenized_docs)

    # Ergebnisse
    #print("Themenzuweisung:", topics)
    #print("Wahrscheinlichkeiten:", probs)

    str_topic_with_probs = []
    #for t, p in zip(topics, probs):
    #    str_topic_with_probs.append(str(t) + ": " + str(p))

    print(i, "documents were empty!")   
    return topic_model.get_topics()

print("Done")


Done


In [ ]:
# import torch
# from transformers import AutoModel, AutoTokenizer

# # Modell und Tokenizer laden
# bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
# tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

# # Anzahl der Songtexte, die verarbeitet werden sollen
# amount = 10  # Beispiel: Anzahl der Texte, die verarbeitet werden sollen

# # INPUT TWEET IST BEREITS NORMALISIERT!
# # Tokenisierung und Padding/Truncation
# lyrics = subdataset_90['lyrics'][:amount]
# inputs = tokenizer(lyrics, return_tensors="pt", padding=True, truncation=True, max_length=512)

# # Verarbeitung der Eingabe durch das Modell
# with torch.no_grad():
#     features = bertweet(**inputs)  # Models outputs sind jetzt Tupel

# # Ausgabe der Features (optional: erster Layer oder hidden states)
# print(features.last_hidden_state)  # Beispielausgabe
# print("Done")


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


NameError: name 'subdataset_90' is not defined

In [11]:
import time
import datetime
from datasets import load_from_disk
import os
def run_analytics_bert_different_Sentence_Transformers(amount_per_iteration,name_of_run, sentence_transformer):

    start_time = time.time()
    amount = amount_per_iteration

    path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Subdatasets/rock/"

    
    rap_70_lyrics_bert = processBertTopics(load_from_disk(path + 'subdataset_70_rock')['lyrics'][:amount],sentence_transformer)
    rap_80_lyrics_bert = processBertTopics(load_from_disk(path + "subdataset_80_rock")['lyrics'][:amount],sentence_transformer)
    rap_90_lyrics_bert = processBertTopics(load_from_disk(path + "subdataset_90_rock")['lyrics'][:amount],sentence_transformer)
    rap_00_lyrics_bert = processBertTopics(load_from_disk(path + "subdataset_00_rock")['lyrics'][:amount],sentence_transformer)
    rap_10_lyrics_bert = processBertTopics(load_from_disk(path + "subdataset_10_rock")['lyrics'][:amount],sentence_transformer)
    rap_20_lyrics_bert = processBertTopics(load_from_disk(path + "subdataset_20_rock")['lyrics'][:amount],sentence_transformer)
    
    
    

    list_of_lyrics_bert = {
        "70s": rap_70_lyrics_bert,
        "80s": rap_80_lyrics_bert,
        "90s": rap_90_lyrics_bert,
        "00s": rap_00_lyrics_bert,
        "10s": rap_10_lyrics_bert,
        "20s": rap_20_lyrics_bert,
    }
    end_time = time.time()

    amount_of_time = end_time - start_time
    
    data_for_csv_bert = []
    
    for decade_name, decade in list_of_lyrics_bert.items():
        i = 0
        
        for topics in decade:
            i = i + 1
            data_for_csv_bert.append({
                "Method": "BERT",
                "Genre": "Rock",
                "Decade": decade_name,
                "Topic": str(decade[topics])
            })
    data_for_csv_bert.append({
                "Method": "TIME",
                "Genre": "TIME",
                "Decade": "TIME",
                "Topic": str(amount_of_time)
            })
    
    # Write to csv
    now = datetime.datetime.now()
    path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Analysis_Folder/BERT/Rock/"
    
    # Datei schreiben
    name_of_file = "topics_rock_BERT_" + "6"+":"+ "21"+ ":50" + "_MiniLM.csv"
    output_file = os.path.join(path, name_of_file)
    

    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Method", "Genre", "Decade", "Topic"])
        writer.writeheader()  
        writer.writerows(data_for_csv_bert) 
   
    print(f"CSV-Datei '{output_file}' wurde erfolgreich erstellt!")
print("Done")


Done


In [12]:
run_analytics_bert_different_Sentence_Transformers(1000,"BERT_mit_stop_words","all-MiniLM-L6-v2")

Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.


/usr/local/lib/python3.10/dist-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


0 documents were empty!
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
0 documents were empty!
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
0 documents were empty!
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
0 documents were empty!
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
0 documents were empty!
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
0 documents were empty!
CSV-Datei '/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Analysis_Folder/BERT/Rock/topics_rock_BERT_6:21:50_MiniLM.csv' wurde erfolgreich erstellt!


In [87]:
topic_model.get_topics(0)

NameError: name 'topic_model' is not defined